In [1]:
from enrichr import Enrichr
import pandas as pd
import os
import glob
from tqdm import tqdm
def find_files(directory, substring, extension):
    # 存储符合条件的文件路径
    matching_files = []
    # 遍历给定目录及其所有子目录
    for root, dirs, files in os.walk(directory):
        # 检查每个文件
        for file in files:
            # 检查文件名是否包含特定子字符串并且以特定扩展名结尾
            if substring in file and file.endswith(extension):
                # 如果符合条件，添加完整路径到列表
                matching_files.append(os.path.join(root, file))
    return matching_files

In [2]:
en = Enrichr(library = ['KEGG_2021_Human', 'GO_Cellular_Component_2023', 'GO_Biological_Process_2023', 'GO_Molecular_Function_2023'])
en = Enrichr(cutoff = 0.25)

In [8]:
files = find_files('/data/twang15/spatial_protein/chip_cytometry/chip_cytometry_1/spatial_func', 'svg', '.tsv')
files[0].split('/')[7]

'Reg001'

In [12]:
for file in tqdm(files):
    tmp = pd.read_csv(file, sep='\t')
    tmp = tmp.head(int(len(tmp) * 0.01) if len(tmp) > 10000 else 200)
    en.addList(tmp['Gene'].tolist())
    tissue = 'Colon'
    reg = file.split('/')[7]
    dataset = 'SCP_CC1'
    tech = 'CC'
    results = en.enrich()
    results = pd.DataFrame(results)
    os.makedirs(f'/data/twang15/spatial_protein/chip_cytometry/chip_cytometry_1/spatial_pathway/{reg}/', exist_ok=True)
    results[results['Pval'] < 0.05]
    expanded_results = results.assign(Genes=results['Genes'].str.split(';')).explode('Genes')
    expanded_results = expanded_results.head(int(len(expanded_results) * 0.1))
    expanded_results.set_index('Genes', inplace=True)
    expanded_results['Technology'] = tech
    expanded_results['Dataset'] = dataset
    expanded_results['Tissue'] = tissue
    expanded_results['Region'] = reg
    expanded_results.to_csv(f'/data/twang15/spatial_protein/chip_cytometry/chip_cytometry_1/spatial_pathway/{reg}/results.txt', sep='\t', index_label='Genes')

100%|██████████| 21/21 [00:24<00:00,  1.17s/it]
